# Load your own PyTorch BERT model

In the previous [example](https://github.com/deepjavalibrary/djl/blob/master/jupyter/BERTQA.ipynb), you run BERT inference with the model from Model Zoo. You can also load the model on your own pre-trained BERT and use custom classes as the input and output.

In general, the PyTorch BERT model from [HuggingFace](https://github.com/huggingface/transformers) requires these three inputs:

- word indices: The index of each word in a sentence
- word types: The type index of the word.
- attention mask: The mask indicates to the model which tokens should be attended to, and which should not after batching sequence together.

We will dive deep into these details later.

## Preparation

In [1]:
import $ivy.`ai.djl:api:0.17.0`
import $ivy.`ai.djl.pytorch:pytorch-model-zoo:0.17.0`
import $ivy.`ai.djl.pytorch:pytorch-engine:0.17.0`
import $ivy.`org.slf4j:slf4j-api:1.7.36`
import $ivy.`org.slf4j:slf4j-simple:1.7.36`

import $ivy.$                  

import $ivy.$                                        

import $ivy.$                                     

import $ivy.$                           

import $ivy.$                              

## Import java packages

In [2]:
import java.io.IOException
import java.nio.file.Paths
import java.util

import ai.djl.modality.nlp.DefaultVocabulary
import ai.djl.modality.nlp.bert.BertTokenizer
import ai.djl.modality.nlp.qa.QAInput
import ai.djl.ndarray.NDList
import ai.djl.repository.zoo.Criteria
import ai.djl.training.util.{DownloadUtils, ProgressBar}
import ai.djl.translate.{Batchifier, Translator, TranslatorContext}

import java.io.IOException

import java.nio.file.Paths

import java.util


import ai.djl.modality.nlp.DefaultVocabulary

import ai.djl.modality.nlp.bert.BertTokenizer

import ai.djl.modality.nlp.qa.QAInput

import ai.djl.ndarray.NDList

import ai.djl.repository.zoo.Criteria

import ai.djl.training.util.{DownloadUtils, ProgressBar}

import ai.djl.translate.{Batchifier, Translator, TranslatorContext}

## Download model and vocab files

In [3]:
DownloadUtils.download(
  "https://djl-ai.s3.amazonaws.com/mlrepo/model/nlp/question_answer/ai/djl/pytorch/bertqa/0.0.1/bert-base-uncased-vocab.txt.gz",
  "build/pytorch/bert-qa/vocab.txt",
  new ProgressBar
)

DownloadUtils.download(
  "https://djl-ai.s3.amazonaws.com/mlrepo/model/nlp/question_answer/ai/djl/pytorch/bertqa/0.0.1/trace_bertqa.pt.gz",
  "build/pytorch/bert-qa/bert-qa.pt",
  new ProgressBar
)

## Create translator

In [4]:
class BertQuAnTranslator extends Translator[QAInput, String] {

  private var vocabulary: DefaultVocabulary = _
  private var tokenizer: BertTokenizer = _
  private var tokenList: util.List[String] = _

  override def prepare(ctx: TranslatorContext): Unit = {
    val path = Paths.get("build/pytorch/bert-qa/vocab.txt")
    vocabulary = DefaultVocabulary.builder
      .optMinFrequency(1)
      .addFromTextFile(path)
      .optUnknownToken("[UNK]")
      .build
    tokenizer = new BertTokenizer()
  }

  override def processInput(ctx: TranslatorContext, input: QAInput): NDList = {
    val token = tokenizer.encode(input.getQuestion.toLowerCase, input.getParagraph.toLowerCase, 384)
    // get the encoded tokens that would be used in precessOutput
    tokenList = token.getTokens
    // map the tokens(String) to indices(long)

    val manager = ctx.getNDManager
    val indices = tokenList.stream().mapToLong(vocabulary.getIndex).toArray
    val attentionMask = token.getAttentionMask.stream().mapToLong(i => i).toArray
    val tokenType = token.getTokenTypes.stream().mapToLong(i => i).toArray
    val indicesArray = manager.create(indices)
    val attentionMaskArray = manager.create(attentionMask)
    val tokenTypeArray = manager.create(tokenType)

    new NDList(indicesArray, attentionMaskArray, tokenTypeArray)
  }

  override def processOutput(ctx: TranslatorContext, list: NDList): String = {
    val startLogits = list.get(0)
    val endLogits = list.get(1)
    val startIdx = startLogits.argMax().getLong().asInstanceOf[Int]
    val endIdx = endLogits.argMax().getLong().asInstanceOf[Int]
    tokenList.subList(startIdx, endIdx + 1).toString
  }

  override def getBatchifier: Batchifier = Batchifier.STACK
}


defined class BertQuAnTranslator

## Load pytorch model

In [5]:
val translator = new BertQuAnTranslator()
val criteria = Criteria.builder
  .setTypes(classOf[QAInput], classOf[String])
  .optModelPath(Paths.get("build/pytorch/bert-qa/"))
  .optTranslator(translator)
  .optProgress(new ProgressBar)
  .build

val model = criteria.loadModel()
val predictor = model.newPredictor(translator)

Loading:     100% |████████████████████████████████████████|


[scala-interpreter-1] INFO ai.djl.pytorch.engine.PtEngine - Number of inter-op threads is 4
[scala-interpreter-1] INFO ai.djl.pytorch.engine.PtEngine - Number of intra-op threads is 4


translator: BertQuAnTranslator = ammonite.$sess.cmd3$Helper$BertQuAnTranslator@221769bc
criteria: Criteria[QAInput, String] = Criteria:
	Application: UNDEFINED
	Input: class ai.djl.modality.nlp.qa.QAInput
	Output: class java.lang.String
	ModelZoo: ai.djl.localmodelzoo

model: ai.djl.repository.zoo.ZooModel[QAInput, String] = ai.djl.repository.zoo.ZooModel@7a1ac919
predictor: ai.djl.inference.Predictor[QAInput, String] = ai.djl.inference.Predictor@547dbfd

## Test model

In [6]:
val question = "When did BBC Japan start broadcasting?";
val context = "BBC Japan was a general entertainment Channel.\n" +
    "Which operated between December 2004 and April 2006.\n" +
    "It ceased operations after its Japanese distributor folded.";
val input = new QAInput(question, context);
val predictResult = predictor.predict(input)

question: String = "When did BBC Japan start broadcasting?"
context: String = """BBC Japan was a general entertainment Channel.
Which operated between December 2004 and April 2006.
It ceased operations after its Japanese distributor folded."""
input: QAInput = ai.djl.modality.nlp.qa.QAInput@2cb677a9
predictResult: String = "[december, 2004]"

**A little more complicated example**

In [7]:
val context = """The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries."""

context: String = "The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (\"Norman\" comes from \"Norseman\") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries."

In [8]:
val q1 = "When were the Normans in Normandy?"
val q2 = "In what country is Normandy located?"

q1: String = "When were the Normans in Normandy?"
q2: String = "In what country is Normandy located?"

In [9]:
predictor.predict(new QAInput(q1, context))

res8: String = "[10th, and, 11th, centuries]"

In [10]:
predictor.predict(new QAInput(q2, context))

res9: String = "[france]"

In [11]:
predictor.close()